In [1]:
%reload_ext autoreload
%autoreload 2

This notebook runs part of the GENIES generalisation eval. We miss the train part

TODO 
- [ ] run it on the [uploaded](https://huggingface.co/genies-models?search_models=7b) GENIES models
- [ ] group by category then radar plot
- [ ] why is it missing `score.weight` and why are results almost random??



In [2]:
from datasets import load_dataset

from open_pref_eval.evaluation import evaluate_models
# from open_pref_eval.helpers.load_models import load_peft_model

see 
- https://github.com/Joshuaclymer/GENIES
- https://github.com/wassname/GENIES/blob/main/nbs/01_mjc_convert_data_to_preference.ipynb

In [3]:
GENIES = [
    {"source": "code_easy", "target": "code_hard", "label": "extreme", "category": "difficulty"},
    {"source": "us_history_textbook", "target": "us_history_fiction", "label": "extreme", "category": "context"},
    {"source": "raven_matrices", "target": "us_history", "label": "extreme", "category": "skill"},
    {"source": "alpaca_mmlu", "target": "spanish_output", "label": "extreme", "category": "encoding"},
    {"source": "math", "target": "change_my_view", "label": "extreme", "category": "skill"},
    {"source": "alpaca_easy", "target": "alpaca_hard", "label": "extreme", "category": "difficulty"},
    {"source": "alpaca_mmlu", "target": "raven_matrices", "label": "extreme", "category": "pretraining_similarity"},
    {"source": "alpaca_mmlu", "target": "ranking_logic", "label": "extreme", "category": "pretraining_similarity"},
    {"source": "alpaca_low_quality", "target": "alpaca_high_quality", "label": "extreme", "category": "quality"},
    {"source": "alpaca_short", "target": "alpaca_long", "target_reference": "alpaca_mmlu", "label": "extreme", "category": "spurious_cues"},
    {"source": "alpaca_mmlu", "target": "wrong_arc", "label": "probing", "category": "spurious_cues"},
    {"source": "alpaca_mmlu", "target": "truthful_qa", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "sycophancy_mimicry", "target_reference": "quote_attribution", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "survival_influence", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "reward_seeking", "label": "probing", "category": "unwanted_personas"}
]

In [4]:
N = 128

DEBUG = True
if DEBUG:
    N = 32

In [5]:
datasets = []
for row in GENIES:
    name = row['target']
    try:
        ds = load_dataset('wassname/genie_dpo', name=name, split=f'test[:{N}]', keep_in_memory=False)
        datasets.append(ds)
    except ValueError:
        print(f"Dataset {name} not found")
datasets

[Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 32
 }),
 Dataset({
     features: ['prom

In [6]:
!mkdir -p ./models
!pip install sentencepiece -q


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# see https://github.com/Joshuaclymer/GENIES/blob/22c8afb2551851fb3f2d1a2dcf70e7608908f6b1/src/interventions/lora_fine_tune/eval.py#L25
quantization_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.float32,

    load_in_8bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.float16,
)
model_kwargs=dict(
    # torch_dtype=torch.float16,
    
    # attn_implementation="flash_attention_2",
    low_cpu_mem_usage=True,
    quantization_config=quantization_config,
    
)

trainer_kwargs = dict(
    # trl.DPOTrainer args
    # f16=True,
    # f16_full_eval=True,
    per_device_eval_batch_size=2,
    torch_empty_cache_steps=100,
)


In [8]:
# download base model

import os
from pathlib import Path




def download_from_hf(model_id, output_dir, use_fast=True):
    if os.path.exists(f"{output_dir}/pytorch_model.bin"):
        print("Model already downloaded. Exiting...")
        return
    print("Downloading from HuggingFace...")
    tokenizer = AutoTokenizer.from_pretrained(
        model_id, 
        use_fast=use_fast, 
        trust_remote_code=True)

    tokenizer.save_pretrained(output_dir)

    model = AutoModelForCausalLM.from_pretrained(
        model_id, trust_remote_code=True, **model_kwargs)

    os.makedirs(output_dir, exist_ok=True)

    model.save_pretrained(output_dir)

base_model = Path("./models/llama-7b")
if not base_model.exists():
    download_from_hf('NousResearch/Llama-2-7b-hf', "./models/llama-7b")

In [9]:
model_names = [
    'genies-models/llama-7b-code_easy',
    'genies-models/llama-7b-us_history_textbook',
    'genies-models/llama-7b-alpaca_mmlu',
    'genies-models/llama-7b-alpaca_easy',
    'genies-models/llama-7b-raven_matrices',
    'genies-models/llama-7b-math',
]

In [10]:


if DEBUG:
    model_names = model_names[:2]
    datasets = datasets[:2]

In [11]:
# FIXME: I need to load with peft
from tqdm.auto import tqdm
from open_pref_eval.evaluation import evaluate_model
from open_pref_eval.helpers.mem import clear_mem
import pandas as pd
from transformers import AutoModelForSequenceClassification
from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM

base_model_name = 'NousResearch/Llama-2-7b-hf'

tokenizer = AutoTokenizer.from_pretrained(base_model_name, 
                                        #   use_fast=False,
                                            trust_remote_code=True)
if tokenizer.pad_token_id == None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, 
                                                                return_dict=True, 
                                                                num_labels=2, 
                                                                **model_kwargs)

from peft import prepare_model_for_kbit_training
base_model = prepare_model_for_kbit_training(base_model)
model_name = model_names[0]
adapter_name = model_name.split('-')[-1]
model = PeftModelForCausalLM.from_pretrained(base_model, model_name, adapter_name=adapter_name, is_trainable=False, **model_kwargs).to('cuda')

model.eval();

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at NousResearch/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/media/wassname/SGIronWolf/projects5/elk/open_pref_eval/.venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (code_easy): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (code_easy): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (code_easy): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): M

In [12]:
# load all adapter onto model
for model_name in tqdm(model_names[1:], unit='model'):
    adapter_name = model_name.split('-')[-1]
    model.load_adapter(model_name, adapter_name)
    model.set_adapter(adapter_name)


  0%|          | 0/1 [00:00<?, ?model/s]

In [13]:
# from peft.peft_model import load_peft_weights
# weights={}
# for model_name in model_names:
#         weights[model_name] = adapters_weights = load_peft_weights(model_name, device='cpu')

## Yes they are diff
# weights[model_names[0]]['base_model.model.model.layers.31.self_attn.q_proj.lora_B.weight']
# weights[model_names[1]]['base_model.model.model.layers.31.self_attn.q_proj.lora_B.weight']

In [ ]:
torch.set_float32_matmul_precision('highest')

In [14]:
# from peft.peft_model import BaseTunerLayer, PeftModel
# from peft.utils.other import ModulesToSaveWrapper
# def adapter_is_active(model: PeftModel) -> bool:
#     """Given a peft model work out is adapters are enabled or disabled"""
#     for module in model.model.modules():
#         if isinstance(module, (BaseTunerLayer, ModulesToSaveWrapper)):
#             # print(help(module.enable_adapters))
#             return module._disable_adapters

# adapter_is_active(model)

In [15]:
adapters = list(model.peft_config.keys())
adapters

# adapter_names = [None] +list(model.peft_config.keys())

['code_easy', 'us_history_textbook']

In [16]:
from open_pref_eval.trainer import OPETrainer, OPEConfig
from typing import Tuple, Optional, Dict, Any

class OPETrainerCls(OPETrainer):

    def get_batch_logps(
        self,
        logits: torch.FloatTensor,
        labels: torch.LongTensor,
        label_pad_token_id: int = -100,
        is_encoder_decoder: bool = False,
    ) -> Tuple[torch.FloatTensor, torch.LongTensor]:
        """
        We modify this to return the logps and mask without reducing them
        """
        # if logits.shape[:-1] != labels.shape:
        #     raise ValueError("Logits (batch and sequence length dim) and labels must have the same shape.")

        if not is_encoder_decoder:
            labels = labels[:, 1:].clone()
            # logits = logits[:, :-1, :]
        loss_mask = labels != label_pad_token_id

        # dummy token; we'll ignore the losses on these tokens later
        labels[labels == label_pad_token_id] = 0

        # per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)
        per_token_logps = torch.log_softmax(logits, -1)

        return per_token_logps, loss_mask


    def concatenated_forward(
        self, model, batch):
        """
        We modify this to simply return the logps and mask without reducing them
        """

        concatenated_batch = self.concatenated_inputs(
            batch,
            is_encoder_decoder=self.is_encoder_decoder,
            is_vision_model=self.is_vision_model,
            label_pad_token_id=self.label_pad_token_id,
            padding_value=self.padding_value,
            device=self.accelerator.device,
        )
        len_chosen = batch["chosen_labels"].shape[0]

        model_kwargs = {}

        if self.is_encoder_decoder:
            model_kwargs["labels"] = concatenated_batch["concatenated_labels"]
            model_kwargs["decoder_input_ids"] = concatenated_batch.pop("concatenated_decoder_input_ids", None)

        from peft import PeftModelForSequenceClassification
        assert isinstance(model, (PeftModelForSequenceClassification, AutoModelForSequenceClassification))
        outputs = model(
            concatenated_batch["concatenated_input_ids"],
            attention_mask=concatenated_batch["concatenated_attention_mask"],
            use_cache=False,
            **model_kwargs,
        )
        nans = 1-torch.isfinite(outputs.logits).float().mean()
        assert torch.isfinite(outputs.logits).all(), f"Got nans in logits {nans}"
        all_logits = outputs.logits[:, 0][:, None] # convert to logits by taking last class

        per_token_logps, mask = self.get_batch_logps(
            logits=all_logits,
            labels=concatenated_batch["concatenated_labels"],
            # average_log_prob=self.loss_type == "ipo",
            label_pad_token_id=self.label_pad_token_id,
            is_encoder_decoder=self.is_encoder_decoder,
        )

        chosen_t_logps = per_token_logps[:len_chosen]
        rejected_t_logps = per_token_logps[len_chosen:]

        chosen_mask = mask[:len_chosen]
        rejected_mask = mask[len_chosen:]

        return (chosen_t_logps, rejected_t_logps, chosen_mask, rejected_mask)
    


def score_cls(logp_c, logp_r, mask_c, mask_r):
    """in this case we just take in classification outputs."""
    s =  logp_c / (logp_c + logp_r)
    return s.squeeze(1)

In [17]:
# # TODO make sure adapter give diff results
# ds = datasets[0]
# for adapter in adapters:
#     print(adapter)
#     results = evaluate_model(model=model, datasets=[ds], adapter_name=adapter)
#     print(results)

In [18]:
# assert isinstance(model, AutoModelForSequenceClassification)
# model

In [19]:
from open_pref_eval.trainer import dummy_dataset
from open_pref_eval.evaluation import alias_trl_kwargs
import tempfile

def get_dummy_trainer(model=None, tokenizer=None, model_name:Optional[str]=None, per_device_eval_batch_size=8, model_kwargs={}, **kwargs):
    """
    Make a dummy trainer, 

    For keyword arguments, see 
    - [transformers.TrainingArguments](https://huggingface.co/docs/transformers/v4.43.3/en/main_classes/trainer#transformers.TrainingArguments)
    - [trl.DPOConfig](https://huggingface.co/docs/trl/main/en/dpo_trainer#trl.DPOConfig)

    """
    with tempfile.TemporaryDirectory() as tmp_dir:
        training_args = OPEConfig(
            output_dir=tmp_dir,
            per_device_eval_batch_size=per_device_eval_batch_size,
            **kwargs
        )

    if model_name is not None:
        model = AutoModelForCausalLM.from_pretrained(model_name,
                                                    **model_kwargs,
                                                     )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

    if model is None:
        raise ValueError('model or model_name must be provided')

    # we use a TRL class
    trainer = OPETrainerCls(
        model=model,
        ref_model=None,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=dummy_dataset,
        eval_dataset=dummy_dataset,
    )
    return trainer

trainer_kwargs2 = alias_trl_kwargs(trainer_kwargs)
trainer = get_dummy_trainer(model=model, tokenizer=tokenizer, model_kwargs=model_kwargs, **trainer_kwargs2)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [20]:

_, df_raw = evaluate_model(
datasets=datasets, 
trainer=trainer,
# **trainer_kwargs, 
# model_kwargs=model_kwargs, 
score_fn=score_cls)
# __annotations__

display(_)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Eval genie_dpo-code_hard-test[:32]:   0%|          | 0/8 [00:00<?, ?it/s]

Error: Got nans in logits 0.625


AssertionError: Got nans in logits 0.625

In [ ]:
df_raw['prob']

In [ ]:
_

In [ ]:
1/0

In [ ]:
from pathlib import Path
import pickle
f_cache = Path(f"./.cache/peft_{model_name}_results_raw.pkl")

In [ ]:
from open_pref_eval.scoring import score_1st_diverg, score_cumsum, score_dpo, score_ipo, score_weighted


if f_cache.exists() and not DEBUG:
    print("Loading from cache")
    dfs = pickle.load(open(f_cache, 'rb'))
else:
    score_fns = [score_1st_diverg, score_weighted, score_cumsum, score_dpo, score_ipo]
    dfs = {}
    for score_fn in tqdm(score_fns, unit='score_fn'):

        _, df_raw = evaluate_model(
            datasets=datasets, 
            model=model, 
            tokenizer=tokenizer, 
            **trainer_kwargs, 
            model_kwargs=model_kwargs, 
            score_fn=score_fn)
        _
        dfs[score_fn.__name__] = df_raw

        print(score_fn.__name__)
        display(_)
    pickle.dump(dfs, open(f_cache, 'wb'))

In [ ]:
f_cache.parent.mkdir(parents=True, exist_ok=True)


In [ ]:
# how much diff in terms of prob?
# FIXME this is uncalibrated so kind of pointless
# TODO try calibrating
# for k, v in dfs.items():
#     print(k)
#     vv = v['prob']
#     # print(vv.min(), vv.max(), vv.mean(), vv.median(), vv.std())
#     print(f"min={vv.min():.2f} max={vv.max():.2f} mean={vv.mean():.2f} median={vv.median():.2f} std={vv.std():.2f}")
#     std_by_ds = vv.reset_index().groupby('dataset').prob.mean().std()
#     std_by_adapter =vv.reset_index().groupby('adapter').prob.mean().std()
#     print(f"std_by_ds: {std_by_ds:.2g}, std_by_adapter: {std_by_adapter:.2g}")

In [ ]:
# diff in acc? each model doesn't vary much why is that?
for k, v in dfs.items():
    print(k)
    vv = v['correct']
    print(f"min={vv.min():.2f} max={vv.max():.2f} mean={vv.mean():.2f} median={vv.median():.2f} std={vv.std():.2f}")
    std_by_ds = vv.reset_index().groupby('dataset').correct.mean().std()
    std_by_adapter =vv.reset_index().groupby('adapter').correct.mean().std()
    print(f"std_by_ds: {std_by_ds:.2g}, std_by_adapter: {std_by_adapter:.2g}")

In [ ]:
# what about per questions by model
for k, v in dfs.items():
    vv = v.correct.reset_index()
    pass
vv

In [ ]:
# plot them
for k, v in dfs.items():
    # v.plot(kind='bar', title=k, figsize=(20, 10))
    print(f"# {k}")
    print(f"Mean acc {v['correct'].mean():2.2g}")
    x = v.loc['genie_dpo-code_hard-test[:64]']['correct']
    d = (x-x['code_easy'])['us_history_textbook']
    print(f"Code should be better than history at code {d:2.2g}<0?")

    x = v.loc['genie_dpo-us_history_fiction-test[:64]']['correct']
    d = (x-x['us_history_textbook'])['code_easy']
    print(f"history model should be better than code model at history {d:2.2g}<0?")

    print()
# v['correct']

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['dataset', 'adapter'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res

In [ ]:
   
# dfs = []
# dfs_raw = []
# for model_name in tqdm(model_names, unit='model'):
#     adapter_name = model_name.split('-')[-1]
#     # Hmm perhaps we need to free mem
#     # for prev_adapter_name in model.peft_config.keys():
#         # print(f"Deleting {prev_adapter_name}")
#         # model.delete_adapter(prev_adapter_name)
#     clear_mem()
    
#     model.load_adapter(model_name, adapter_name)
#     model.set_adapter(adapter_name)
#     # tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#     df_agg, df_raw = evaluate_model(datasets=datasets, model=model, tokenizer=tokenizer, **trainer_kwargs, model_kwargs=model_kwargs,)
#     df_agg['model'] = model_name
#     df_raw['model'] = model_name
#     dfs.append(df_agg)
#     dfs_raw.append(df_raw)
#     print(model_name)
#     display(df_agg)
# df_agg = pd.concat(dfs)
# df_raw = pd.concat(dfs_raw)

In [ ]:
# model

In [ ]:
# AutoPeftModelForCausalLM.from_pretrained(base_model=model, model_name=model_name)

In [ ]:
# df_agg, df_raw = evaluate_models(model_names=model_names, datasets=datasets,
                                
#                                 # trl.DPOTrainer args
#                                 bf16=True,
#                                 bf16_full_eval=True,
#                                 per_device_eval_batch_size=3,
#                                 torch_empty_cache_steps=100,

#                                 # transformers.AutoModelForCausalLM args
#                                 model_kwargs=model_kwargs,
# )

In [ ]:
# import pandas as pd
# from tqdm.auto import tqdm
# from open_pref_eval.evaluation import evaluate_model, get_dummy_trainer, clear_mem

# def evaluate_models2(datasets, model_names, **kwargs):
#     dfs = []
#     dfs_raw = []
#     for model_name in tqdm(model_names, unit='model'):
#         model = AutoModelForCausalLM.from_pretrained(model_name)
#         tokenizer = AutoTokenizer.from_pretrained(model_name)# TODO bnb
#         trainer = get_dummy_trainer(model=model, tokenizer=tokenizer, **kwargs)
#         df_agg, df_raw = evaluate_model(datasets, trainer)
#         clear_mem()
#         dfs.append(df_agg)
#         dfs_raw.append(df_raw)
#     df_agg = pd.concat(dfs)
#     df_raw = pd.concat(dfs_raw)
#     return df_agg, dfs_raw

In [ ]:
# df_agg, df_raw = evaluate_models2(model_names=model_names, datasets=datasets,
                                
#                                 # trl args
#                                 bf16=True,
#                                 bf16_full_eval=True,
#                                 per_device_eval_batch_size=2,
#                                 torch_empty_cache_steps=100,
# )
df_raw

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['dataset', 'adapter'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res

In [ ]:
df_res['genies-models/llama-7b-raven_matrices']
df_res.loc['genie_dpo-raven_matrices-train'] # shouldn't the raven mode be better at raven??



In [ ]:
df_res.loc['genie_dpo-us_history-train'] # should the history one be best? maybe there is a bug here?

In [ ]:
rename = {}
for row in GENIES:
    s = 'genie_dpo-'+row['target']+'-train'
    rename[s] = row['category']
df_raw['category'] = df_raw.dataset.replace(rename)

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['category', 'model'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res